In [ ]:
import os
import math
import numpy as np
from scipy import signal, stats
from scipy.fft import fftshift
import matplotlib.pyplot as plt
import statistics
import time
import pickle
import json
import datetime
import warnings
warnings.filterwarnings('ignore')
import librosa
import soundfile as sf
import pandas as pd
import plotly.graph_objs as go
import plotly.offline as pyo
import seaborn as sns # Seaborn (version : 0.11.1)
plt.style.use('seaborn')
indices_escogidos=['BETA','P','M','NP','BNF','MD','SF','RMS','CF','SC','SNR']
from sklearn import metrics
import os
import math
import numpy as np
from scipy import signal, stats
from scipy.fft import fftshift
import matplotlib.pyplot as plt
import statistics
import time
import pickle
import json
import datetime
import warnings
warnings.filterwarnings('ignore')
import librosa
import soundfile as sf
import pandas as pd
import plotly.graph_objs as go
import plotly.offline as pyo
import seaborn as sns # Seaborn (version : 0.11.1)
plt.style.use('seaborn')
indices_escogidos=['BETA','P','M','NP','BNF','MD','SF','RMS','CF','SC','SNR']
from sklearn import metrics
from numpy import linalg as LA
from scipy.spatial import distance
from sklearn.cluster import kmeans_plusplus
from sklearn.datasets import make_blobs
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.mixture import GaussianMixture
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from copy import deepcopy
from matplotlib.pyplot import figure
import pandas as pd
from scipy.stats import kruskal, spearmanr
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
def calculate_correlation_kruskal(df_merged,buffer_values_df, columns_to_correlate, columns_to_correlate_with):
    correlation_df = pd.DataFrame(index=columns_to_correlate, columns=columns_to_correlate_with, dtype=float)
    h_value_df = pd.DataFrame(index=columns_to_correlate, columns=columns_to_correlate_with, dtype=float)
    p_value_df = pd.DataFrame(index=columns_to_correlate, columns=columns_to_correlate_with, dtype=float)

    for i in columns_to_correlate:
        for j in columns_to_correlate_with:
            # Get the values from df_merged and buffer_values_df for columns i and j
            values_i = df_merged[i]
            values_j = buffer_values_df[j]

            # Calculate Spearman rank correlation coefficient
            correlation_coefficient, _ = spearmanr(values_i, values_j)
            correlation_df.at[i, j] = round(correlation_coefficient,3)

            # Perform Kruskal-Wallis test
            h_value, p_value = kruskal(values_i, values_j)
            h_value_df.at[i, j] = h_value
            p_value_df.at[i, j] = p_value

    return correlation_df, h_value_df, p_value_df

In [ ]:
grab_17_20=pd.read_csv("/content/drive/MyDrive/Doctorado/2023/Agosto/grab_17_20.csv")
grab_20_5=pd.read_csv("/content/drive/MyDrive/Doctorado/2023/Agosto/grab_20_5.csv")
grab_5_8=pd.read_csv("/content/drive/MyDrive/Doctorado/2023/Agosto/grab_5_8.csv")
grab_8_17=pd.read_csv("/content/drive/MyDrive/Doctorado/2023/Agosto/grab_8_17.csv")

In [ ]:
buffer=pd.read_csv("/content/drive/MyDrive/Doctorado/2023/Agosto/buffer.csv")
buffer = buffer.rename(columns={'field_numb': 'site'})

In [ ]:
grab_8_17

,file,ZCR,MEANt,VARt,SKEWt,KURTt,LEQt,BGNt,SNRt,MED,...,H_pairedShannon,H_gamma,H_GiniSimpson,RAOQ,AGI,ROItotal,ROIcover,fecha,hora,site
0,//home/vmartinezarias/Documents/Zamuro/Ultraso...,31372.933333,-6.887912e-19,0.000193,0.177502,5.834632,55.877214,-28.298142,1.478258,0.038467,...,5.518148,21751.811409,0.972104,0.309954,1.254270,2203,2.307533,20220907,15,RZUH04
1,//home/vmartinezarias/Documents/Zamuro/Ultraso...,40403.816667,-5.919069e-19,0.000172,0.221258,14.253758,55.379005,-28.342884,2.038076,0.038270,...,6.050323,27827.025409,0.990705,0.346173,1.249948,2024,1.740025,20220912,13,RZUH04
2,//home/vmartinezarias/Documents/Zamuro/Ultraso...,36570.916667,5.251972e-19,0.000259,0.217033,11.059525,57.146265,-26.752327,2.233526,0.045960,...,6.068535,26735.547136,0.990190,0.407788,1.291630,2505,3.563665,20220904,12,RZUH04
3,//home/vmartinezarias/Documents/Zamuro/Ultraso...,36388.666667,-1.752071e-20,0.000183,0.209978,11.078873,55.643758,-29.224443,2.327447,0.034576,...,5.442487,22030.994684,0.962471,0.376591,1.269506,2556,3.189569,20220908,12,RZUH04
4,//home/vmartinezarias/Documents/Zamuro/Ultraso...,39789.500000,-7.408618e-19,0.000185,0.241936,11.325877,55.680847,-28.050052,1.964332,0.039582,...,6.028208,26967.730832,0.990549,0.334746,1.246427,2039,1.969150,20220912,12,RZUH04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21952,//home/vmartinezarias/Documents/Zamuro/Ultraso...,42833.066667,-4.985210e-19,0.000102,0.305256,7.816180,53.088560,-33.296468,3.586555,0.021636,...,5.507842,19264.965935,0.983057,0.206480,1.314274,931,8.707384,20220907,12,RZUG09
21953,//home/vmartinezarias/Documents/Zamuro/Ultraso...,34867.183333,8.608083e-19,0.000201,0.218835,8.625105,56.044746,-29.860445,2.180966,0.032135,...,5.399764,17320.070268,0.975638,0.296981,1.307240,862,6.544258,20220906,11,RZUG09
21954,//home/vmartinezarias/Documents/Zamuro/Ultraso...,43485.600000,-3.329284e-19,0.000078,1.733379,1311.338835,51.952624,-32.831745,3.106412,0.022825,...,5.903098,25398.430146,0.989651,0.328133,1.272347,1188,5.379848,20220909,12,RZUG09
21955,//home/vmartinezarias/Documents/Zamuro/Ultraso...,48439.200000,3.026418e-19,0.000039,0.181327,5.525403,48.962696,-35.889701,2.077763,0.016051,...,5.880387,29152.685259,0.988572,0.272588,1.266460,800,6.027290,20220908,14,RZUG09


## 08-17

In [ ]:
X_train11 = grab_8_17.groupby('site')[[ 'EPS', 'BGNf', 'Hf', 'NDSI', 'rBA', 'ADI']].mean()
X_train11

,EPS,BGNf,Hf,NDSI,rBA,ADI
site,,,,,,
RZUA01,0.043531,-40.556006,0.678332,0.585586,29.862555,2.058310
RZUA02,0.039387,-49.877965,0.726988,0.301841,1.952216,2.077945
RZUA03a,0.028480,-38.298512,0.772120,0.287502,2.956173,2.077026
RZUA03b,0.048144,-28.898508,0.761147,0.103458,5.289926,1.909331
RZUA04,0.023088,-45.376415,0.743428,0.276384,13.025849,1.977103
...,...,...,...,...,...,...
RZUH09,0.036698,-40.952088,0.719393,0.267395,15.398028,1.784278
RZUH10,0.044417,-47.703069,0.723673,0.113421,6.645519,1.940670
RZUH11,0.037471,-32.691845,0.779820,-0.218930,1.800928,1.895907


In [ ]:
buffer2022=pd.read_excel("/content/drive/MyDrive/Doctorado/2023/octubre/allindices2022.xlsx")
#buffer = buffer.rename(columns={'field_numb': 'site'})
buffer2022[['site', 'XCOORD.y', 'YCOORD.y', 'ndmi2022', 'brillo2022', '_slavimean',
       '_MNDWImean']]
X_train11 = grab_8_17.groupby('site')[[ 'EPS', 'BGNf', 'Hf', 'NDSI', 'rBA', 'ADI']].mean()
rand=5
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(X_train11.values)
X_train11 = pd.DataFrame(scaled_features, index=X_train11.index, columns=X_train11.columns)
GM = GaussianMixture(n_components=3, random_state=rand, covariance_type="full").fit(X_train11)
pred=GM.predict(X_train11)
predproba=GM.predict_proba(X_train11)
X_train11["pred"]=pred
X_train11['component1_prob'] = predproba[:, 0]
X_train11['component2_prob'] = predproba[:, 1]
X_train11['component3_prob'] = predproba[:, 2]
merged_df = pd.merge(X_train11, buffer2022, on="site")
merged_df = pd.merge(merged_df, buffer, on="site")
merged_df = merged_df[merged_df['site'] != 'RZUA02']
merged_df = merged_df[merged_df['site'] != 'RZUE02']
merged_df = merged_df[merged_df['site'] != 'RZUC03']
merged_df = merged_df[merged_df['site'] != 'RZUC07']
merged_df = merged_df[merged_df['site'] != 'RZUG05']
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(merged_df.iloc[:,1:].values)
scaled_features_df = pd.DataFrame(scaled_features, index=merged_df.iloc[:,1:].index, columns=merged_df.iloc[:,1:].columns)
merged_df.iloc[:,1:]=scaled_features_df

merged_df

,site,EPS,BGNf,Hf,NDSI,rBA,ADI,pred,component1_prob,component2_prob,...,llanos200_b200mean_200,llanos200_b200median_200,ICHN250_b250mean_250,ICHN250_b250median_250,FM200_b200mean_200,FM200_b200median_200,ICHN200_b200mean_200,ICHN200_b200median_200,ICHN100_b100mean_100,ICHN100_b100median_100
0,RZUA01,0.201559,0.795690,0.396394,0.856743,0.136398,0.968042,1.0,0.000000e+00,4.774825e-19,...,0.890478,0.969209,0.825867,0.943928,0.842404,1.000000,0.876312,0.957621,0.983765,0.989564
2,RZUA03b,0.229023,1.000000,0.645420,0.531611,0.023077,0.741072,0.5,1.795726e-03,9.982043e-01,...,0.814446,0.964755,0.815549,0.942094,0.701164,1.000000,0.779726,0.922898,0.722209,0.779019
3,RZUA04,0.079847,0.711207,0.592141,0.648227,0.058753,0.844323,0.5,2.555215e-68,9.723429e-01,...,0.804035,0.943957,0.772022,0.876821,0.642006,1.000000,0.719122,0.789548,0.531630,0.499752
4,RZUA05,0.164172,0.692352,0.659997,0.810665,0.381191,0.940204,1.0,0.000000e+00,8.839818e-238,...,0.917130,0.975337,0.806451,0.877083,0.806481,1.000000,0.789717,0.881927,0.708318,0.742850
5,RZUA06,0.095921,0.703240,0.564290,0.820789,0.063857,0.962548,1.0,5.985902e-70,2.373482e-02,...,0.892473,0.964753,0.695380,0.821832,0.764833,1.000000,0.747241,0.852033,0.805048,0.863099
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,RZUH09,0.160877,0.788748,0.519864,0.642165,0.069692,0.550553,0.5,4.420883e-91,1.000000e+00,...,0.641660,0.950434,0.626057,0.711630,0.610908,1.000000,0.661228,0.785346,0.754908,0.839823
90,RZUH10,0.206835,0.670429,0.532734,0.538330,0.029329,0.788817,0.5,3.777273e-17,9.999998e-01,...,0.489070,0.460560,0.374921,0.212293,0.204277,0.000000,0.310535,0.202993,0.210764,0.164337
91,RZUH11,0.165480,0.933518,0.701571,0.314204,0.006987,0.720620,0.0,9.834164e-01,1.658362e-02,...,0.447945,0.491368,0.213594,0.173708,0.013876,0.000000,0.200375,0.170276,0.247308,0.283583
92,RZUH12,0.375657,0.795595,0.475682,0.389639,0.014932,0.625593,0.5,2.620412e-06,9.999974e-01,...,0.576444,0.612136,0.305479,0.306816,0.038400,0.000000,0.324104,0.322671,0.369220,0.338752


In [ ]:
merged_df['ICHN2022']=merged_df['COB100_b100mean_100']+merged_df['DI100_b100mean_100']+merged_df['FM100_b100mean_100']+merged_df['SLAVI100_b100mean_100']+merged_df['_MNDWImean']+merged_df['DN100_b100mean_100']+merged_df['SL100_b100mean_100']+merged_df['AD100_b100mean_100']

In [ ]:
merged_df

,site,EPS,BGNf,Hf,NDSI,rBA,ADI,pred,component1_prob,component2_prob,...,llanos200_b200median_200,ICHN250_b250mean_250,ICHN250_b250median_250,FM200_b200mean_200,FM200_b200median_200,ICHN200_b200mean_200,ICHN200_b200median_200,ICHN100_b100mean_100,ICHN100_b100median_100,ICHN2022
0,RZUA01,0.201559,0.795690,0.396394,0.856743,0.136398,0.968042,1.0,0.000000e+00,4.774825e-19,...,0.969209,0.825867,0.943928,0.842404,1.000000,0.876312,0.957621,0.983765,0.989564,6.409862
2,RZUA03b,0.229023,1.000000,0.645420,0.531611,0.023077,0.741072,0.5,1.795726e-03,9.982043e-01,...,0.964755,0.815549,0.942094,0.701164,1.000000,0.779726,0.922898,0.722209,0.779019,4.829493
3,RZUA04,0.079847,0.711207,0.592141,0.648227,0.058753,0.844323,0.5,2.555215e-68,9.723429e-01,...,0.943957,0.772022,0.876821,0.642006,1.000000,0.719122,0.789548,0.531630,0.499752,4.269625
4,RZUA05,0.164172,0.692352,0.659997,0.810665,0.381191,0.940204,1.0,0.000000e+00,8.839818e-238,...,0.975337,0.806451,0.877083,0.806481,1.000000,0.789717,0.881927,0.708318,0.742850,4.524839
5,RZUA06,0.095921,0.703240,0.564290,0.820789,0.063857,0.962548,1.0,5.985902e-70,2.373482e-02,...,0.964753,0.695380,0.821832,0.764833,1.000000,0.747241,0.852033,0.805048,0.863099,4.747561
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,RZUH09,0.160877,0.788748,0.519864,0.642165,0.069692,0.550553,0.5,4.420883e-91,1.000000e+00,...,0.950434,0.626057,0.711630,0.610908,1.000000,0.661228,0.785346,0.754908,0.839823,5.353195
90,RZUH10,0.206835,0.670429,0.532734,0.538330,0.029329,0.788817,0.5,3.777273e-17,9.999998e-01,...,0.460560,0.374921,0.212293,0.204277,0.000000,0.310535,0.202993,0.210764,0.164337,2.862457
91,RZUH11,0.165480,0.933518,0.701571,0.314204,0.006987,0.720620,0.0,9.834164e-01,1.658362e-02,...,0.491368,0.213594,0.173708,0.013876,0.000000,0.200375,0.170276,0.247308,0.283583,3.121909
92,RZUH12,0.375657,0.795595,0.475682,0.389639,0.014932,0.625593,0.5,2.620412e-06,9.999974e-01,...,0.612136,0.305479,0.306816,0.038400,0.000000,0.324104,0.322671,0.369220,0.338752,3.852229


In [ ]:
cd /content/drive/MyDrive/Doctorado/2024

/content/drive/MyDrive/Doctorado/2024


In [ ]:
merged_df.to_csv("Todoslosindices08.csv", index=False)

In [ ]:
pwd

'/content/drive/MyDrive/Doctorado/2024'

In [ ]:
columns_to_correlate= ['component1_prob', 'component2_prob', 'component3_prob']
columns_to_correlate_with=['ICHN2022','FM100_b100mean_100',  'DN100_b100mean_100',
      'DI100_b100mean_100', 'AD100_b100mean_100',  'COB100_b100mean_100','brillo100_b100mean_100',
      '_slavimean', 'SL100_b100mean_100', 'ndmi2022','_MNDWImean']

correlation_df, h_value_df, p_value_df=calculate_correlation_kruskal(merged_df,merged_df, columns_to_correlate, columns_to_correlate_with)

correlation_df

,ICHN2022,FM100_b100mean_100,DN100_b100mean_100,DI100_b100mean_100,AD100_b100mean_100,COB100_b100mean_100,brillo100_b100mean_100,_slavimean,SL100_b100mean_100,ndmi2022,_MNDWImean
component1_prob,-0.721,-0.680,-0.357,-0.424,-0.705,-0.659,0.635,-0.655,0.089,-0.678,0.269
component2_prob,-0.304,-0.354,-0.138,-0.181,-0.360,-0.329,0.286,-0.338,0.095,-0.371,0.303
component3_prob,0.710,0.710,0.361,0.435,0.753,0.687,-0.669,0.668,-0.163,0.671,-0.416


In [ ]:
columns_to_correlate= ['component1_prob', 'component2_prob', 'component3_prob']
columns_to_correlate_with=[ 'EPS', 'BGNf', 'Hf', 'NDSI', 'rBA', 'ADI']

correlation_df, h_value_df, p_value_df=calculate_correlation_kruskal(merged_df,merged_df, columns_to_correlate, columns_to_correlate_with)

correlation_df

,EPS,BGNf,Hf,NDSI,rBA,ADI
component1_prob,-0.073,-0.012,0.062,-0.916,-0.934,-0.638
component2_prob,-0.100,-0.106,0.612,-0.436,-0.478,-0.100
component3_prob,-0.079,-0.007,-0.005,0.941,0.880,0.780


## 05-08

In [ ]:
buffer2022=pd.read_excel("/content/drive/MyDrive/Doctorado/2023/octubre/allindices2022.xlsx")
#buffer = buffer.rename(columns={'field_numb': 'site'})
buffer2022[['site', 'XCOORD.y', 'YCOORD.y', 'ndmi2022', 'brillo2022', '_slavimean',
       '_MNDWImean']]
X_train11 = grab_5_8.groupby('site')[[ 'EPS', 'BGNf', 'Hf', 'NDSI', 'rBA', 'ADI']].mean()
rand=4
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(X_train11.values)
X_train11 = pd.DataFrame(scaled_features, index=X_train11.index, columns=X_train11.columns)
GM = GaussianMixture(n_components=3, random_state=rand, covariance_type="full").fit(X_train11)
pred=GM.predict(X_train11)
predproba=GM.predict_proba(X_train11)
X_train11["pred"]=pred
X_train11['component1_prob'] = predproba[:, 0]
X_train11['component2_prob'] = predproba[:, 1]
X_train11['component3_prob'] = predproba[:, 2]
merged_df = pd.merge(X_train11, buffer2022, on="site")
merged_df = pd.merge(merged_df, buffer, on="site")
merged_df = merged_df[merged_df['site'] != 'RZUA02']
merged_df = merged_df[merged_df['site'] != 'RZUE02']
merged_df = merged_df[merged_df['site'] != 'RZUC03']
merged_df = merged_df[merged_df['site'] != 'RZUC07']
merged_df = merged_df[merged_df['site'] != 'RZUG05']
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(merged_df.iloc[:,1:].values)
scaled_features_df = pd.DataFrame(scaled_features, index=merged_df.iloc[:,1:].index, columns=merged_df.iloc[:,1:].columns)
merged_df.iloc[:,1:]=scaled_features_df

merged_df

,site,EPS,BGNf,Hf,NDSI,rBA,ADI,pred,component1_prob,component2_prob,...,llanos200_b200mean_200,llanos200_b200median_200,ICHN250_b250mean_250,ICHN250_b250median_250,FM200_b200mean_200,FM200_b200median_200,ICHN200_b200mean_200,ICHN200_b200median_200,ICHN100_b100mean_100,ICHN100_b100median_100
0,RZUA01,0.311468,0.730255,0.528739,0.717314,0.127664,0.934548,1.0,1.120116e-03,2.113434e-03,...,0.890478,0.969209,0.825867,0.943928,0.842404,1.000000,0.876312,0.957621,0.983765,0.989564
2,RZUA03b,0.263525,0.948748,0.649670,0.764536,0.179672,0.748996,1.0,1.463872e-02,4.979253e-05,...,0.814446,0.964755,0.815549,0.942094,0.701164,1.000000,0.779726,0.922898,0.722209,0.779019
3,RZUA04,0.072956,0.672496,0.395495,0.227705,0.053923,0.340177,0.5,2.255388e-39,1.000000e+00,...,0.804035,0.943957,0.772022,0.876821,0.642006,1.000000,0.719122,0.789548,0.531630,0.499752
4,RZUA05,0.246428,0.658236,0.634729,0.797457,0.328003,0.947595,1.0,4.310641e-03,1.739327e-30,...,0.917130,0.975337,0.806451,0.877083,0.806481,1.000000,0.789717,0.881927,0.708318,0.742850
5,RZUA06,0.148846,0.704182,0.481105,0.722228,0.189341,0.895192,1.0,8.470633e-04,1.684652e-09,...,0.892473,0.964753,0.695380,0.821832,0.764833,1.000000,0.747241,0.852033,0.805048,0.863099
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,RZUH09,0.298787,0.937145,0.161296,1.000000,0.909095,0.775563,0.0,1.000000e+00,1.986629e-285,...,0.641660,0.950434,0.626057,0.711630,0.610908,1.000000,0.661228,0.785346,0.754908,0.839823
87,RZUH10,0.349511,0.645851,0.706800,0.640033,0.068939,0.855340,0.5,5.195566e-07,9.031999e-01,...,0.489070,0.460560,0.374921,0.212293,0.204277,0.000000,0.310535,0.202993,0.210764,0.164337
88,RZUH11,0.369825,0.883983,0.600015,0.843703,0.251065,0.771463,1.0,3.135431e-02,8.533573e-13,...,0.447945,0.491368,0.213594,0.173708,0.013876,0.000000,0.200375,0.170276,0.247308,0.283583
89,RZUH12,0.080360,0.829237,0.324062,0.300157,0.123008,0.000000,0.5,2.076943e-50,1.000000e+00,...,0.576444,0.612136,0.305479,0.306816,0.038400,0.000000,0.324104,0.322671,0.369220,0.338752


In [ ]:
merged_df['ICHN2022']=merged_df['COB100_b100mean_100']+merged_df['DI100_b100mean_100']+merged_df['FM100_b100mean_100']+merged_df['SLAVI100_b100mean_100']+merged_df['_MNDWImean']+merged_df['DN100_b100mean_100']+merged_df['SL100_b100mean_100']+merged_df['AD100_b100mean_100']

In [ ]:
merged_df

,site,EPS,BGNf,Hf,NDSI,rBA,ADI,pred,component1_prob,component2_prob,...,llanos200_b200median_200,ICHN250_b250mean_250,ICHN250_b250median_250,FM200_b200mean_200,FM200_b200median_200,ICHN200_b200mean_200,ICHN200_b200median_200,ICHN100_b100mean_100,ICHN100_b100median_100,ICHN2022
0,RZUA01,0.311468,0.730255,0.528739,0.717314,0.127664,0.934548,1.0,1.120116e-03,2.113434e-03,...,0.969209,0.825867,0.943928,0.842404,1.000000,0.876312,0.957621,0.983765,0.989564,6.409862
2,RZUA03b,0.263525,0.948748,0.649670,0.764536,0.179672,0.748996,1.0,1.463872e-02,4.979253e-05,...,0.964755,0.815549,0.942094,0.701164,1.000000,0.779726,0.922898,0.722209,0.779019,4.829493
3,RZUA04,0.072956,0.672496,0.395495,0.227705,0.053923,0.340177,0.5,2.255388e-39,1.000000e+00,...,0.943957,0.772022,0.876821,0.642006,1.000000,0.719122,0.789548,0.531630,0.499752,4.269625
4,RZUA05,0.246428,0.658236,0.634729,0.797457,0.328003,0.947595,1.0,4.310641e-03,1.739327e-30,...,0.975337,0.806451,0.877083,0.806481,1.000000,0.789717,0.881927,0.708318,0.742850,4.524839
5,RZUA06,0.148846,0.704182,0.481105,0.722228,0.189341,0.895192,1.0,8.470633e-04,1.684652e-09,...,0.964753,0.695380,0.821832,0.764833,1.000000,0.747241,0.852033,0.805048,0.863099,4.747561
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,RZUH09,0.298787,0.937145,0.161296,1.000000,0.909095,0.775563,0.0,1.000000e+00,1.986629e-285,...,0.950434,0.626057,0.711630,0.610908,1.000000,0.661228,0.785346,0.754908,0.839823,5.353195
87,RZUH10,0.349511,0.645851,0.706800,0.640033,0.068939,0.855340,0.5,5.195566e-07,9.031999e-01,...,0.460560,0.374921,0.212293,0.204277,0.000000,0.310535,0.202993,0.210764,0.164337,2.862457
88,RZUH11,0.369825,0.883983,0.600015,0.843703,0.251065,0.771463,1.0,3.135431e-02,8.533573e-13,...,0.491368,0.213594,0.173708,0.013876,0.000000,0.200375,0.170276,0.247308,0.283583,3.121909
89,RZUH12,0.080360,0.829237,0.324062,0.300157,0.123008,0.000000,0.5,2.076943e-50,1.000000e+00,...,0.612136,0.305479,0.306816,0.038400,0.000000,0.324104,0.322671,0.369220,0.338752,3.852229


In [ ]:
columns_to_correlate= ['component1_prob', 'component2_prob', 'component3_prob']
columns_to_correlate_with=['ICHN2022','FM100_b100mean_100',  'DN100_b100mean_100',
      'DI100_b100mean_100', 'AD100_b100mean_100',  'COB100_b100mean_100','brillo100_b100mean_100',
      '_slavimean', 'SL100_b100mean_100', 'ndmi2022','_MNDWImean']

correlation_df, h_value_df, p_value_df=calculate_correlation_kruskal(merged_df,merged_df, columns_to_correlate, columns_to_correlate_with)

correlation_df

,ICHN2022,FM100_b100mean_100,DN100_b100mean_100,DI100_b100mean_100,AD100_b100mean_100,COB100_b100mean_100,brillo100_b100mean_100,_slavimean,SL100_b100mean_100,ndmi2022,_MNDWImean
component1_prob,0.440,0.395,0.181,0.217,0.406,0.409,-0.384,0.409,-0.027,0.402,-0.268
component2_prob,-0.441,-0.405,-0.144,-0.172,-0.414,-0.386,0.357,-0.456,-0.011,-0.457,0.262
component3_prob,0.020,0.040,-0.102,-0.065,0.100,-0.015,0.043,0.070,-0.047,0.052,-0.152


In [ ]:
columns_to_correlate= ['component1_prob', 'component2_prob', 'component3_prob']
columns_to_correlate_with=[ 'EPS', 'BGNf', 'Hf', 'NDSI', 'rBA', 'ADI']

correlation_df, h_value_df, p_value_df=calculate_correlation_kruskal(merged_df,merged_df, columns_to_correlate, columns_to_correlate_with)

correlation_df

,EPS,BGNf,Hf,NDSI,rBA,ADI
component1_prob,0.108,0.450,-0.712,0.966,0.886,0.209
component2_prob,-0.077,-0.426,0.714,-0.894,-0.934,-0.198
component3_prob,0.194,-0.055,0.147,0.057,0.021,0.020


In [ ]:

\begin{table}[H]
\centering
\rotatebox{90}{%
\begin{tabular}{|l|l|l|}
\hline
\textbf{Cluster} & \textbf{Correlations} & \textbf{Summary} \\
\hline
C1 &  Strong correlation Hf: (0.64), NDSI (-0.76)& \\
   & NDMI: Negative correlation (-0.308) & \\
\hline
C2 &  Positive correlation HF(-0.61), NDSI(0.71) &  High Active behavior in most coordinates \\
   & \quad \hspace{1.3cm}  &  \\
\hline
C3 & &  \\
   &  Positive correlation NDSI (0.88), Hf (-0.68)  & Hf Complementary relationship with C1 \\
   & &  \\
\hline
\end{tabular} }
\caption{Summary of each cluster prototype 05 to 08}
\label{tab:cluster_summary}
\end{table}



## 17_20

In [ ]:
buffer2022=pd.read_excel("/content/drive/MyDrive/Doctorado/2023/octubre/allindices2022.xlsx")
#buffer = buffer.rename(columns={'field_numb': 'site'})
buffer2022[['site', 'XCOORD.y', 'YCOORD.y', 'ndmi2022', 'brillo2022', '_slavimean',
       '_MNDWImean']]
X_train11 = grab_17_20.groupby('site')[[ 'EPS', 'BGNf', 'Hf', 'NDSI', 'rBA', 'ADI']].mean()
rand=6
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(X_train11.values)
X_train11 = pd.DataFrame(scaled_features, index=X_train11.index, columns=X_train11.columns)
GM = GaussianMixture(n_components=4, random_state=rand, covariance_type="full").fit(X_train11)
pred=GM.predict(X_train11)
predproba=GM.predict_proba(X_train11)
X_train11["pred"]=pred
X_train11['component1_prob'] = predproba[:, 0]
X_train11['component2_prob'] = predproba[:, 1]
X_train11['component3_prob'] = predproba[:, 2]
X_train11['component4_prob'] = predproba[:, 3]
merged_df = pd.merge(X_train11, buffer2022, on="site")
merged_df = pd.merge(merged_df, buffer, on="site")
merged_df = merged_df[merged_df['site'] != 'RZUA02']
merged_df = merged_df[merged_df['site'] != 'RZUE02']
merged_df = merged_df[merged_df['site'] != 'RZUC03']
merged_df = merged_df[merged_df['site'] != 'RZUC07']
merged_df = merged_df[merged_df['site'] != 'RZUG05']
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(merged_df.iloc[:,1:].values)
scaled_features_df = pd.DataFrame(scaled_features, index=merged_df.iloc[:,1:].index, columns=merged_df.iloc[:,1:].columns)
merged_df.iloc[:,1:]=scaled_features_df

merged_df

,site,EPS,BGNf,Hf,NDSI,rBA,ADI,pred,component1_prob,component2_prob,...,llanos200_b200mean_200,llanos200_b200median_200,ICHN250_b250mean_250,ICHN250_b250median_250,FM200_b200mean_200,FM200_b200median_200,ICHN200_b200mean_200,ICHN200_b200median_200,ICHN100_b100mean_100,ICHN100_b100median_100
0,RZUA01,0.404125,0.850989,0.365941,0.865244,0.179335,0.944801,0.333333,3.557790e-01,6.356658e-01,...,0.890478,0.969209,0.825867,0.943928,0.842404,1.000000,0.876312,0.957621,0.983765,0.989564
2,RZUA03b,0.305558,0.968510,0.612553,0.673343,0.059999,0.582738,0.666667,3.560696e-15,3.250842e-20,...,0.814446,0.964755,0.815549,0.942094,0.701164,1.000000,0.779726,0.922898,0.722209,0.779019
3,RZUA04,0.150517,0.748042,0.444299,0.435453,0.066269,0.443870,0.333333,3.875702e-38,7.986921e-01,...,0.804035,0.943957,0.772022,0.876821,0.642006,1.000000,0.719122,0.789548,0.531630,0.499752
4,RZUA05,0.319203,0.845379,0.537350,0.864061,0.281405,0.887643,0.000000,9.928715e-01,7.124927e-03,...,0.917130,0.975337,0.806451,0.877083,0.806481,1.000000,0.789717,0.881927,0.708318,0.742850
5,RZUA06,0.223484,0.754574,0.459444,0.834757,0.108291,0.906481,0.333333,4.470676e-03,8.254768e-01,...,0.892473,0.964753,0.695380,0.821832,0.764833,1.000000,0.747241,0.852033,0.805048,0.863099
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,RZUH09,0.247980,0.919845,0.406393,0.772162,0.204940,0.580228,0.666667,2.269344e-15,1.097702e-05,...,0.641660,0.950434,0.626057,0.711630,0.610908,1.000000,0.661228,0.785346,0.754908,0.839823
90,RZUH10,0.334350,0.735526,0.443674,0.635678,0.099342,0.626142,0.333333,5.747152e-13,9.216405e-01,...,0.489070,0.460560,0.374921,0.212293,0.204277,0.000000,0.310535,0.202993,0.210764,0.164337
91,RZUH11,0.391050,0.975542,0.557441,0.447205,0.062978,0.609478,0.666667,1.637909e-14,5.401618e-18,...,0.447945,0.491368,0.213594,0.173708,0.013876,0.000000,0.200375,0.170276,0.247308,0.283583
92,RZUH12,0.406124,0.850168,0.448041,0.145931,0.016640,0.438149,0.666667,3.125393e-37,2.832720e-13,...,0.576444,0.612136,0.305479,0.306816,0.038400,0.000000,0.324104,0.322671,0.369220,0.338752


In [ ]:
merged_df['ICHN2022']=merged_df['COB100_b100mean_100']+merged_df['DI100_b100mean_100']+merged_df['FM100_b100mean_100']+merged_df['SLAVI100_b100mean_100']+merged_df['_MNDWImean']+merged_df['DN100_b100mean_100']+merged_df['SL100_b100mean_100']+merged_df['AD100_b100mean_100']

In [ ]:
merged_df

,site,EPS,BGNf,Hf,NDSI,rBA,ADI,pred,component1_prob,component2_prob,...,llanos200_b200median_200,ICHN250_b250mean_250,ICHN250_b250median_250,FM200_b200mean_200,FM200_b200median_200,ICHN200_b200mean_200,ICHN200_b200median_200,ICHN100_b100mean_100,ICHN100_b100median_100,ICHN2022
0,RZUA01,0.404125,0.850989,0.365941,0.865244,0.179335,0.944801,0.333333,3.557790e-01,6.356658e-01,...,0.969209,0.825867,0.943928,0.842404,1.000000,0.876312,0.957621,0.983765,0.989564,6.409862
2,RZUA03b,0.305558,0.968510,0.612553,0.673343,0.059999,0.582738,0.666667,3.560696e-15,3.250842e-20,...,0.964755,0.815549,0.942094,0.701164,1.000000,0.779726,0.922898,0.722209,0.779019,4.829493
3,RZUA04,0.150517,0.748042,0.444299,0.435453,0.066269,0.443870,0.333333,3.875702e-38,7.986921e-01,...,0.943957,0.772022,0.876821,0.642006,1.000000,0.719122,0.789548,0.531630,0.499752,4.269625
4,RZUA05,0.319203,0.845379,0.537350,0.864061,0.281405,0.887643,0.000000,9.928715e-01,7.124927e-03,...,0.975337,0.806451,0.877083,0.806481,1.000000,0.789717,0.881927,0.708318,0.742850,4.524839
5,RZUA06,0.223484,0.754574,0.459444,0.834757,0.108291,0.906481,0.333333,4.470676e-03,8.254768e-01,...,0.964753,0.695380,0.821832,0.764833,1.000000,0.747241,0.852033,0.805048,0.863099,4.747561
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,RZUH09,0.247980,0.919845,0.406393,0.772162,0.204940,0.580228,0.666667,2.269344e-15,1.097702e-05,...,0.950434,0.626057,0.711630,0.610908,1.000000,0.661228,0.785346,0.754908,0.839823,5.353195
90,RZUH10,0.334350,0.735526,0.443674,0.635678,0.099342,0.626142,0.333333,5.747152e-13,9.216405e-01,...,0.460560,0.374921,0.212293,0.204277,0.000000,0.310535,0.202993,0.210764,0.164337,2.862457
91,RZUH11,0.391050,0.975542,0.557441,0.447205,0.062978,0.609478,0.666667,1.637909e-14,5.401618e-18,...,0.491368,0.213594,0.173708,0.013876,0.000000,0.200375,0.170276,0.247308,0.283583,3.121909
92,RZUH12,0.406124,0.850168,0.448041,0.145931,0.016640,0.438149,0.666667,3.125393e-37,2.832720e-13,...,0.612136,0.305479,0.306816,0.038400,0.000000,0.324104,0.322671,0.369220,0.338752,3.852229


In [ ]:
columns_to_correlate= ['component1_prob', 'component2_prob', 'component3_prob', 'component4_prob']
columns_to_correlate_with=['ICHN2022','FM100_b100mean_100',  'DN100_b100mean_100',
      'DI100_b100mean_100', 'AD100_b100mean_100',  'COB100_b100mean_100','brillo100_b100mean_100',
      '_slavimean', 'SL100_b100mean_100', 'ndmi2022','_MNDWImean']

correlation_df, h_value_df, p_value_df=calculate_correlation_kruskal(merged_df,merged_df, columns_to_correlate, columns_to_correlate_with)

correlation_df

,ICHN2022,FM100_b100mean_100,DN100_b100mean_100,DI100_b100mean_100,AD100_b100mean_100,COB100_b100mean_100,brillo100_b100mean_100,_slavimean,SL100_b100mean_100,ndmi2022,_MNDWImean
component1_prob,0.647,0.625,0.280,0.346,0.663,0.598,-0.561,0.585,-0.003,0.589,-0.326
component2_prob,0.093,0.146,-0.141,-0.080,0.145,0.093,-0.067,0.149,-0.273,0.150,-0.009
component3_prob,-0.473,-0.445,-0.223,-0.254,-0.488,-0.413,0.378,-0.367,0.011,-0.361,0.156
component4_prob,-0.282,-0.277,-0.112,-0.155,-0.268,-0.255,0.214,-0.308,0.004,-0.345,0.110


In [ ]:
columns_to_correlate= ['component1_prob', 'component2_prob', 'component3_prob', 'component4_prob']
columns_to_correlate_with=[ 'EPS', 'BGNf', 'Hf', 'NDSI', 'rBA', 'ADI']

correlation_df, h_value_df, p_value_df=calculate_correlation_kruskal(merged_df,merged_df, columns_to_correlate, columns_to_correlate_with)

correlation_df

,EPS,BGNf,Hf,NDSI,rBA,ADI
component1_prob,0.135,0.003,0.071,-0.560,-0.497,-0.745
component2_prob,0.369,0.334,-0.515,0.621,0.647,0.125
component3_prob,-0.195,0.239,-0.251,0.860,0.777,0.662
component4_prob,-0.235,-0.155,0.743,-0.450,-0.600,0.085


In [ ]:

\begin{table}[H]
\centering
\rotatebox{90}{%
\begin{tabular}{|l|l|l|}
\hline
\textbf{Cluster} & \textbf{Correlations} & \textbf{Summary} \\
\hline
C1 & Strong correlation  Hf (-0.56), NDSI (0.85)  & Similar to cluster 4 in high-value points \\
   & Strong positive correlation SLAVI (0.545), ICHN (0.529), FM (0.521) NDMI (0.54)& \\
\hline
C2 & Hf: Weak positive correlation with Acoustic  and ecological indices &  \\
\hline
C3 & Positive correlations Hf (-0.58), NDSI (0.74) & Dissimilar high-intensity  sites with Cluster 4  \\
   &  positive correlation SLAVI (0.42), COB (0.43), FM (0.42), ICHN (0.43) & \\
\hline
C4 &  high correlation Hf (0.59), NDSI (-0.96), rBA (-0.97)  & \\
   & High correlation COB(-0.50), SLAVI (-0.50), NDMI (-0.50), ICHN(-0.51)  \\
\hline
\end{tabular}}
\caption{Summary of each cluster based on provided information period 17-20}
\label{tab:cluster_summary17-20}
\end{table}


SyntaxError: ignored

## 20_05

In [ ]:
buffer2022=pd.read_excel("/content/drive/MyDrive/Doctorado/2023/octubre/allindices2022.xlsx")
#buffer = buffer.rename(columns={'field_numb': 'site'})
buffer2022[['site', 'XCOORD.y', 'YCOORD.y', 'ndmi2022', 'brillo2022', '_slavimean',
       '_MNDWImean']]
X_train11 = grab_20_5.groupby('site')[[ 'EPS', 'BGNf', 'Hf', 'NDSI', 'rBA', 'ADI']].mean()
rand=2
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(X_train11.values)
X_train11 = pd.DataFrame(scaled_features, index=X_train11.index, columns=X_train11.columns)
GM = GaussianMixture(n_components=5, random_state=rand, covariance_type="full").fit(X_train11)
pred=GM.predict(X_train11)
predproba=GM.predict_proba(X_train11)
X_train11["pred"]=pred
X_train11['component1_prob'] = predproba[:, 0]
X_train11['component2_prob'] = predproba[:, 1]
X_train11['component3_prob'] = predproba[:, 2]
X_train11['component4_prob'] = predproba[:, 3]
X_train11['component5_prob'] = predproba[:, 4]
merged_df = pd.merge(X_train11, buffer2022, on="site")
merged_df = pd.merge(merged_df, buffer, on="site")
merged_df = merged_df[merged_df['site'] != 'RZUA02']
merged_df = merged_df[merged_df['site'] != 'RZUE02']
merged_df = merged_df[merged_df['site'] != 'RZUC03']
merged_df = merged_df[merged_df['site'] != 'RZUC07']
merged_df = merged_df[merged_df['site'] != 'RZUG05']
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(merged_df.iloc[:,1:].values)
scaled_features_df = pd.DataFrame(scaled_features, index=merged_df.iloc[:,1:].index, columns=merged_df.iloc[:,1:].columns)
merged_df.iloc[:,1:]=scaled_features_df

merged_df

,site,EPS,BGNf,Hf,NDSI,rBA,ADI,pred,component1_prob,component2_prob,...,llanos200_b200mean_200,llanos200_b200median_200,ICHN250_b250mean_250,ICHN250_b250median_250,FM200_b200mean_200,FM200_b200median_200,ICHN200_b200mean_200,ICHN200_b200median_200,ICHN100_b100mean_100,ICHN100_b100median_100
0,RZUA01,0.634790,0.719829,0.481607,0.882619,0.133091,0.977477,0.25,1.284894e-07,9.982855e-01,...,0.890478,0.969209,0.825867,0.943928,0.842404,1.000000,0.876312,0.957621,0.983765,0.989564
1,RZUA03b,0.373796,0.671062,0.571069,0.855042,0.109166,0.624285,1.00,5.089189e-05,7.620476e-08,...,0.814446,0.964755,0.815549,0.942094,0.701164,1.000000,0.779726,0.922898,0.722209,0.779019
2,RZUA04,0.272410,0.616288,0.411314,0.410628,0.050146,0.249103,0.50,1.385732e-21,1.361584e-47,...,0.804035,0.943957,0.772022,0.876821,0.642006,1.000000,0.719122,0.789548,0.531630,0.499752
3,RZUA05,0.507187,0.904525,0.461370,0.945150,0.223084,0.898692,0.25,1.713261e-03,9.982879e-01,...,0.917130,0.975337,0.806451,0.877083,0.806481,1.000000,0.789717,0.881927,0.708318,0.742850
4,RZUA06,0.409956,0.627035,0.502230,0.873204,0.157556,0.919999,1.00,4.237894e-05,3.849786e-03,...,0.892473,0.964753,0.695380,0.821832,0.764833,1.000000,0.747241,0.852033,0.805048,0.863099
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,RZUH09,0.373829,0.917784,0.414706,0.995920,0.450943,0.868419,0.00,9.992385e-01,7.614918e-04,...,0.641660,0.950434,0.626057,0.711630,0.610908,1.000000,0.661228,0.785346,0.754908,0.839823
83,RZUH10,0.469499,0.640437,0.412797,0.848560,0.139455,0.589646,1.00,2.362628e-05,1.389249e-02,...,0.489070,0.460560,0.374921,0.212293,0.204277,0.000000,0.310535,0.202993,0.210764,0.164337
84,RZUH11,0.842703,0.768204,0.268876,0.923101,0.241847,0.588368,0.25,4.010588e-08,9.983234e-01,...,0.447945,0.491368,0.213594,0.173708,0.013876,0.000000,0.200375,0.170276,0.247308,0.283583
85,RZUH12,0.351673,0.837998,0.177809,0.382232,0.045160,0.301827,0.50,1.663010e-24,1.990788e-109,...,0.576444,0.612136,0.305479,0.306816,0.038400,0.000000,0.324104,0.322671,0.369220,0.338752


In [ ]:
merged_df['ICHN2022']=merged_df['COB100_b100mean_100']+merged_df['DI100_b100mean_100']+merged_df['FM100_b100mean_100']+merged_df['SLAVI100_b100mean_100']+merged_df['_MNDWImean']+merged_df['DN100_b100mean_100']+merged_df['SL100_b100mean_100']+merged_df['AD100_b100mean_100']

In [ ]:
merged_df

,site,EPS,BGNf,Hf,NDSI,rBA,ADI,pred,component1_prob,component2_prob,...,llanos200_b200median_200,ICHN250_b250mean_250,ICHN250_b250median_250,FM200_b200mean_200,FM200_b200median_200,ICHN200_b200mean_200,ICHN200_b200median_200,ICHN100_b100mean_100,ICHN100_b100median_100,ICHN2022
0,RZUA01,0.634790,0.719829,0.481607,0.882619,0.133091,0.977477,0.25,1.284894e-07,9.982855e-01,...,0.969209,0.825867,0.943928,0.842404,1.000000,0.876312,0.957621,0.983765,0.989564,6.409862
1,RZUA03b,0.373796,0.671062,0.571069,0.855042,0.109166,0.624285,1.00,5.089189e-05,7.620476e-08,...,0.964755,0.815549,0.942094,0.701164,1.000000,0.779726,0.922898,0.722209,0.779019,4.829493
2,RZUA04,0.272410,0.616288,0.411314,0.410628,0.050146,0.249103,0.50,1.385732e-21,1.361584e-47,...,0.943957,0.772022,0.876821,0.642006,1.000000,0.719122,0.789548,0.531630,0.499752,4.269625
3,RZUA05,0.507187,0.904525,0.461370,0.945150,0.223084,0.898692,0.25,1.713261e-03,9.982879e-01,...,0.975337,0.806451,0.877083,0.806481,1.000000,0.789717,0.881927,0.708318,0.742850,4.524839
4,RZUA06,0.409956,0.627035,0.502230,0.873204,0.157556,0.919999,1.00,4.237894e-05,3.849786e-03,...,0.964753,0.695380,0.821832,0.764833,1.000000,0.747241,0.852033,0.805048,0.863099,4.747561
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,RZUH09,0.373829,0.917784,0.414706,0.995920,0.450943,0.868419,0.00,9.992385e-01,7.614918e-04,...,0.950434,0.626057,0.711630,0.610908,1.000000,0.661228,0.785346,0.754908,0.839823,5.353195
83,RZUH10,0.469499,0.640437,0.412797,0.848560,0.139455,0.589646,1.00,2.362628e-05,1.389249e-02,...,0.460560,0.374921,0.212293,0.204277,0.000000,0.310535,0.202993,0.210764,0.164337,2.862457
84,RZUH11,0.842703,0.768204,0.268876,0.923101,0.241847,0.588368,0.25,4.010588e-08,9.983234e-01,...,0.491368,0.213594,0.173708,0.013876,0.000000,0.200375,0.170276,0.247308,0.283583,3.121909
85,RZUH12,0.351673,0.837998,0.177809,0.382232,0.045160,0.301827,0.50,1.663010e-24,1.990788e-109,...,0.612136,0.305479,0.306816,0.038400,0.000000,0.324104,0.322671,0.369220,0.338752,3.852229


In [ ]:
columns_to_correlate= ['component1_prob', 'component2_prob', 'component3_prob', 'component4_prob', 'component5_prob']
columns_to_correlate_with=['ICHN2022','FM100_b100mean_100',  'DN100_b100mean_100',
      'DI100_b100mean_100', 'AD100_b100mean_100',  'COB100_b100mean_100','brillo100_b100mean_100',
      '_slavimean', 'SL100_b100mean_100', 'ndmi2022','_MNDWImean']

correlation_df, h_value_df, p_value_df=calculate_correlation_kruskal(merged_df,merged_df, columns_to_correlate, columns_to_correlate_with)

correlation_df

,ICHN2022,FM100_b100mean_100,DN100_b100mean_100,DI100_b100mean_100,AD100_b100mean_100,COB100_b100mean_100,brillo100_b100mean_100,_slavimean,SL100_b100mean_100,ndmi2022,_MNDWImean
component1_prob,0.440,0.460,0.179,0.199,0.480,0.450,-0.352,0.537,0.029,0.498,-0.352
component2_prob,-0.018,-0.029,-0.197,-0.171,0.014,-0.078,0.070,-0.004,0.144,-0.018,-0.097
component3_prob,-0.479,-0.475,-0.240,-0.243,-0.529,-0.466,0.362,-0.502,-0.007,-0.444,0.258
component4_prob,0.069,0.033,0.119,0.069,0.056,0.052,-0.020,-0.026,-0.010,-0.010,0.105
component5_prob,-0.107,-0.089,-0.024,0.018,-0.140,-0.094,0.075,-0.271,-0.125,-0.219,0.064


In [ ]:
columns_to_correlate= ['component1_prob', 'component2_prob', 'component3_prob', 'component4_prob', 'component5_prob']
columns_to_correlate_with=[ 'EPS', 'BGNf', 'Hf', 'NDSI', 'rBA', 'ADI']

correlation_df, h_value_df, p_value_df=calculate_correlation_kruskal(merged_df,merged_df, columns_to_correlate, columns_to_correlate_with)

correlation_df

,EPS,BGNf,Hf,NDSI,rBA,ADI
component1_prob,-0.046,0.695,-0.249,0.749,0.843,0.420
component2_prob,0.351,0.230,0.192,0.289,0.071,0.089
component3_prob,0.160,-0.382,0.103,-0.640,-0.769,-0.675
component4_prob,-0.266,-0.266,0.266,-0.266,-0.266,0.266
component5_prob,-0.037,-0.449,0.336,-0.266,-0.340,0.016


## Other analysis

In [ ]:

X_train11 = grab_8_17.groupby('site')[[ 'BGNf', 'Hf', 'NDSI', 'rBA', 'ADI']].mean()
rand=5
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(X_train11.values)
X_train11 = pd.DataFrame(scaled_features, index=X_train11.index, columns=X_train11.columns)
GM = GaussianMixture(n_components=3, random_state=rand, covariance_type="full").fit(X_train11)
pred=GM.predict(X_train11)
predproba=GM.predict_proba(X_train11)
X_train11["pred"]=pred
X_train11['component1_prob'] = predproba[:, 0]
X_train11['component2_prob'] = predproba[:, 1]
X_train11['component3_prob'] = predproba[:, 2]
merged_df = pd.merge(X_train11, merged_df, on="site")
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(merged_df.iloc[:,1:].values)
scaled_features_df = pd.DataFrame(scaled_features, index=merged_df.iloc[:,1:].index, columns=merged_df.iloc[:,1:].columns)
merged_df.iloc[:,1:]=scaled_features_df
merged_df
columns_to_correlate= ['component1_prob', 'component2_prob', 'component3_prob']
columns_to_correlate_with=['ICHN2022','FM100_b100mean_100',  'DN100_b100mean_100',
      'DI100_b100mean_100', 'AD100_b100mean_100',  'COB100_b100mean_100','brillo100_b100mean_100',
      '_slavimean', 'SL100_b100mean_100', 'ndmi2022','_MNDWImean']

correlation_df, h_value_df, p_value_df=calculate_correlation_kruskal(merged_df,merged_df, columns_to_correlate, columns_to_correlate_with)
print(correlation_df)

KeyError: ignored

Index(['site', 'XCOORD.y', 'YCOORD.y', 'ndmi2022', 'brillo2022', '_slavimean',
       '_MNDWImean', 'FM', 'DN', 'DI', 'AD', 'COB', 'SL'],
      dtype='object')

In [ ]:
from sklearn.mixture import GaussianMixture
for i in range(0,25):
  X_train11 = grab_8_17.groupby('site')[[ 'BGNf', 'Hf', 'NDSI', 'rBA', 'ADI']].mean()
  rand=4
  scaler = MinMaxScaler()
  scaled_features = scaler.fit_transform(X_train11.values)
  X_train11 = pd.DataFrame(scaled_features, index=X_train11.index, columns=X_train11.columns)
  GM = GaussianMixture(n_components=3, random_state=i, covariance_type="full").fit(X_train11)
  pred=GM.predict(X_train11)
  predproba=GM.predict_proba(X_train11)
  X_train11["pred"]=pred
  X_train11['component1_prob'] = predproba[:, 0]
  X_train11['component2_prob'] = predproba[:, 1]
  X_train11['component3_prob'] = predproba[:, 2]
  merged_df = pd.merge(X_train11, buffer2022, on="site")

  merged_df = merged_df[merged_df['site'] != 'RZUA02']
  merged_df = merged_df[merged_df['site'] != 'RZUE02']
  merged_df = merged_df[merged_df['site'] != 'RZUC03']
  merged_df = merged_df[merged_df['site'] != 'RZUC07']
  merged_df = merged_df[merged_df['site'] != 'RZUG05']
  scaler = MinMaxScaler()
  scaled_features = scaler.fit_transform(merged_df.iloc[:,1:].values)
  scaled_features_df = pd.DataFrame(scaled_features, index=merged_df.iloc[:,1:].index, columns=merged_df.iloc[:,1:].columns)
  merged_df.iloc[:,1:]=scaled_features_df
  merged_df
  columns_to_correlate= ['component1_prob', 'component2_prob', 'component3_prob']
  columns_to_correlate_with=['ndmi2022', 'brillo2022', '_slavimean',
       '_MNDWImean', 'FM', 'DN', 'DI', 'AD', 'COB', 'SL'  ]

  correlation_df, h_value_df, p_value_df=calculate_correlation_kruskal(merged_df,merged_df, columns_to_correlate, columns_to_correlate_with)
  print(correlation_df)

                 ndmi2022  brillo2022  _slavimean  _MNDWImean     FM     DN  \
component1_prob     0.655      -0.267       0.658      -0.439  0.269 -0.031   
component2_prob    -0.675       0.235      -0.651       0.319 -0.255  0.015   
component3_prob    -0.390       0.091      -0.364       0.285 -0.132  0.068   

                    DI     AD    COB     SL  
component1_prob  0.172  0.482  0.286 -0.275  
component2_prob -0.202 -0.487 -0.304  0.218  
component3_prob  0.088 -0.094 -0.093  0.146  
                 ndmi2022  brillo2022  _slavimean  _MNDWImean     FM     DN  \
component1_prob     0.655      -0.267       0.658      -0.439  0.269 -0.031   
component2_prob    -0.392       0.088      -0.366       0.285 -0.131  0.066   
component3_prob    -0.677       0.236      -0.652       0.320 -0.255  0.015   

                    DI     AD    COB     SL  
component1_prob  0.172  0.482  0.286 -0.275  
component2_prob  0.090 -0.096 -0.092  0.145  
component3_prob -0.201 -0.487 -0.304  0.219 

In [ ]:
from sklearn.mixture import GaussianMixture
for i in range(0,25):
  X_train11 = grab_8_17.groupby('site')[[ 'BGNf', 'Hf', 'NDSI', 'rBA', 'ADI']].mean()
  rand=4
  from sklearn.preprocessing import StandardScaler
  from sklearn.preprocessing import MinMaxScaler
  scaler = MinMaxScaler()
  scaled_features = scaler.fit_transform(X_train11.values)
  X_train11 = pd.DataFrame(scaled_features, index=X_train11.index, columns=X_train11.columns)
  GM = GaussianMixture(n_components=3, random_state=i, covariance_type="full").fit(X_train11)
  pred=GM.predict(X_train11)
  predproba=GM.predict_proba(X_train11)
  X_train11["pred"]=pred
  X_train11['component1_prob'] = predproba[:, 0]
  X_train11['component2_prob'] = predproba[:, 1]
  X_train11['component3_prob'] = predproba[:, 2]
  merged_df = pd.merge(X_train11, buffer, on="site")

  merged_df = merged_df[merged_df['site'] != 'RZUA02']
  merged_df = merged_df[merged_df['site'] != 'RZUE02']
  merged_df = merged_df[merged_df['site'] != 'RZUC03']
  merged_df = merged_df[merged_df['site'] != 'RZUC07']
  merged_df = merged_df[merged_df['site'] != 'RZUG05']
  from sklearn.preprocessing import StandardScaler
  from sklearn.preprocessing import MinMaxScaler
  scaler = MinMaxScaler()
  scaled_features = scaler.fit_transform(merged_df.iloc[:,1:].values)
  scaled_features_df = pd.DataFrame(scaled_features, index=merged_df.iloc[:,1:].index, columns=merged_df.iloc[:,1:].columns)
  merged_df.iloc[:,1:]=scaled_features_df
  merged_df
  columns_to_correlate= ['component1_prob', 'component2_prob', 'component3_prob']
  columns_to_correlate_with=['ICHN100_b100mean_100', 'AD100_b100mean_100'  ]

  correlation_df, h_value_df, p_value_df=calculate_correlation_kruskal(merged_df,merged_df, columns_to_correlate, columns_to_correlate_with)
  print(correlation_df)

                 ICHN100_b100mean_100  AD100_b100mean_100
component1_prob                 0.715               0.744
component2_prob                -0.714              -0.726
component3_prob                -0.349              -0.359
                 ICHN100_b100mean_100  AD100_b100mean_100
component1_prob                 0.715               0.744
component2_prob                -0.351              -0.361
component3_prob                -0.715              -0.727
                 ICHN100_b100mean_100  AD100_b100mean_100
component1_prob                 0.715               0.744
component2_prob                -0.349              -0.359
component3_prob                -0.714              -0.726
                 ICHN100_b100mean_100  AD100_b100mean_100
component1_prob                 0.715               0.744
component2_prob                -0.714              -0.726
component3_prob                -0.349              -0.359
                 ICHN100_b100mean_100  AD100_b100mean_100
component1_pro

In [ ]:

  X_train11 = grab_8_17.groupby('site')[[ 'BGNf', 'Hf', 'NDSI', 'rBA', 'ADI']].mean()
  rand=5
  from sklearn.preprocessing import StandardScaler
  from sklearn.preprocessing import MinMaxScaler
  scaler = MinMaxScaler()
  scaled_features = scaler.fit_transform(X_train11.values)
  X_train11 = pd.DataFrame(scaled_features, index=X_train11.index, columns=X_train11.columns)
  GM = GaussianMixture(n_components=3, random_state=rand, covariance_type="full").fit(X_train11)
  pred=GM.predict(X_train11)
  predproba=GM.predict_proba(X_train11)
  X_train11["pred"]=pred
  X_train11['component1_prob'] = predproba[:, 0]
  X_train11['component2_prob'] = predproba[:, 1]
  X_train11['component3_prob'] = predproba[:, 2]
  merged_df = pd.merge(X_train11, buffer, on="site")

  merged_df = merged_df[merged_df['site'] != 'RZUA02']
  merged_df = merged_df[merged_df['site'] != 'RZUE02']
  merged_df = merged_df[merged_df['site'] != 'RZUC03']
  merged_df = merged_df[merged_df['site'] != 'RZUC07']
  merged_df = merged_df[merged_df['site'] != 'RZUG05']
  from sklearn.preprocessing import StandardScaler
  from sklearn.preprocessing import MinMaxScaler
  scaler = MinMaxScaler()
  scaled_features = scaler.fit_transform(merged_df.iloc[:,1:].values)
  scaled_features_df = pd.DataFrame(scaled_features, index=merged_df.iloc[:,1:].index, columns=merged_df.iloc[:,1:].columns)
  merged_df.iloc[:,1:]=scaled_features_df
  merged_df
  columns_to_correlate= ['component1_prob', 'component2_prob', 'component3_prob']
  columns_to_correlate_with=['ICHN100_b100mean_100','FM100_b100mean_100',  'DN100_b100mean_100',
        'DI100_b100mean_100', 'AD100_b100mean_100',  'COB100_b100mean_100','brillo100_b100mean_100',
        'SLAVI100_b100mean_100', 'SL100_b100mean_100', 'NDMI100_b100mean_100','llanos100_b100mean_100']

  correlation_df, h_value_df, p_value_df=calculate_correlation_kruskal(merged_df,merged_df, columns_to_correlate, columns_to_correlate_with)
  print(correlation_df)

                 ICHN100_b100mean_100  FM100_b100mean_100  DN100_b100mean_100  \
component1_prob                -0.715              -0.700              -0.367   
component2_prob                -0.351              -0.355              -0.146   
component3_prob                 0.715               0.701               0.361   

                 DI100_b100mean_100  AD100_b100mean_100  COB100_b100mean_100  \
component1_prob              -0.434              -0.727                -0.68   
component2_prob              -0.191              -0.361                -0.33   
component3_prob               0.433               0.744                 0.68   

                 brillo100_b100mean_100  SLAVI100_b100mean_100  \
component1_prob                   0.649                 -0.690   
component2_prob                   0.299                 -0.380   
component3_prob                  -0.668                  0.688   

                 SL100_b100mean_100  NDMI100_b100mean_100  \
component1_prob             

In [ ]:
correlation_df

,ICHN100_b100mean_100,FM100_b100mean_100,DN100_b100mean_100,DI100_b100mean_100,AD100_b100mean_100,COB100_b100mean_100,brillo100_b100mean_100,SLAVI100_b100mean_100,SL100_b100mean_100,NDMI100_b100mean_100,llanos100_b100mean_100
component1_prob,-0.725,-0.720,-0.364,-0.421,-0.747,-0.701,0.638,-0.684,0.140,-0.688,-0.676
component2_prob,-0.522,-0.524,-0.264,-0.333,-0.556,-0.511,0.466,-0.546,0.233,-0.552,-0.538
component3_prob,0.701,0.701,0.339,0.403,0.743,0.674,-0.642,0.677,-0.171,0.684,0.673


In [ ]:
X_train11

,BGNf,Hf,NDSI,rBA,ADI,pred,component1_prob,component2_prob,component3_prob
site,,,,,,,,,
RZUA01,0.623490,0.496349,0.875996,0.154405,0.998286,2,0.000000e+00,1.650371e-97,1.000000e+00
RZUA02,0.373709,0.652204,0.657621,0.047299,0.998886,1,9.563347e-48,9.963951e-01,3.604857e-03
RZUA03a,0.794964,0.736946,0.624418,0.041203,0.990642,1,1.162151e-37,1.000000e+00,3.950560e-15
RZUA03b,1.000000,0.705350,0.598525,0.037105,0.868612,1,2.413291e-33,1.000000e+00,3.248199e-56
RZUA04,0.535689,0.771754,0.604549,0.038013,0.998769,1,1.426496e-28,1.000000e+00,4.138792e-08
...,...,...,...,...,...,...,...,...,...
RZUH09,0.630467,0.644919,0.678668,0.051767,0.936308,1,9.485060e-75,1.000000e+00,1.745550e-11
RZUH10,0.425385,0.777574,0.594298,0.036483,0.995589,1,4.125495e-29,9.999996e-01,3.799579e-07
RZUH11,0.882750,0.771316,0.298675,0.010888,0.830265,1,1.627683e-08,1.000000e+00,6.420550e-110


In [ ]:
columns_to_correlate= ['component1_prob', 'component2_prob', 'component3_prob']
columns_to_correlate_with=['ICHN100_b100mean_100','FM100_b100mean_100',  'DN100_b100mean_100',
        'DI100_b100mean_100', 'AD100_b100mean_100',  'COB100_b100mean_100','brillo100_b100mean_100',
        'SLAVI100_b100mean_100', 'SL100_b100mean_100', 'NDMI100_b100mean_100','llanos100_b100mean_100']

correlation_df, h_value_df, p_value_df=calculate_correlation_kruskal(merged_df,merged_df, columns_to_correlate, columns_to_correlate_with)
correlation_df

,ICHN100_b100mean_100,FM100_b100mean_100,DN100_b100mean_100,DI100_b100mean_100,AD100_b100mean_100,COB100_b100mean_100,brillo100_b100mean_100,SLAVI100_b100mean_100,SL100_b100mean_100,NDMI100_b100mean_100,llanos100_b100mean_100
component1_prob,-0.725,-0.720,-0.364,-0.421,-0.747,-0.701,0.638,-0.684,0.140,-0.688,-0.676
component2_prob,-0.522,-0.524,-0.264,-0.333,-0.556,-0.511,0.466,-0.546,0.233,-0.552,-0.538
component3_prob,0.701,0.701,0.339,0.403,0.743,0.674,-0.642,0.677,-0.171,0.684,0.673


In [ ]:
X_train11.to_csv("/content/drive/MyDrive/Doctorado/2023/Agosto/Resultados_clusteringMembership/8_17.csv")

In [ ]:
grab_8_17=pd.read_csv("/content/drive/MyDrive/Doctorado/2023/Junio/grab_8_17.csv")

df_agregado = grab_8_17.iloc[:,1:].groupby('site').median()
# Resetear el índice para obtener una columna 'lugar' en el dataframe resultante
df_agregado = df_agregado.reset_index()
# Mostrar el dataframe resultante


In [ ]:
grab_8_17=df_agregado
grab_8_17

,site,EPS,BGNf,Hf,NDSI,rBA,ADI,AEI,fecha,hora,YCOORD.y,XCOORD.y,SLAVI1
0,RZUA01,0.037349,-41.294131,0.681023,0.733510,6.504961,2.079019,0.016325,20220908.0,12.0,1950942.625,4954301.250,0.634330
1,RZUA02,0.041921,-50.266909,0.743658,0.346930,2.062458,2.079167,0.013361,20220907.0,12.0,1950542.624,4954301.249,0.590880
2,RZUA03a,0.022219,-35.134349,0.777714,0.288152,1.809588,2.077133,0.038356,20220903.0,10.0,NaN,NaN,NaN
3,RZUA03b,0.029519,-27.768921,0.765016,0.242314,1.639616,2.047040,0.139949,20220911.0,12.0,1950142.624,4954301.247,0.370052
4,RZUA04,0.009434,-44.448175,0.791702,0.252978,1.677297,2.079138,0.013630,20220908.0,12.0,1949742.624,4954301.246,0.179750
...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,RZUH09,0.030847,-41.043499,0.740730,0.384188,2.247744,2.063734,0.097676,20220908.0,12.0,1947742.623,4957101.251,0.636108
92,RZUH10,0.036735,-48.410575,0.794041,0.234831,1.613803,2.078353,0.026114,20220908.0,12.0,1947342.623,4957101.251,0.134615
93,RZUH11,0.026938,-31.980851,0.791526,-0.288497,0.552196,2.037583,0.156149,20220907.0,12.0,1946942.623,4957101.251,0.247195
94,RZUH12,0.049309,-40.251008,0.719019,-0.071629,0.866324,2.067278,0.085463,20220907.0,12.0,1946542.623,4957101.251,0.262742


In [ ]:
merged_df = pd.merge(grab_8_17, buffer, on="site")

columns_to_correlate= ['NDSI', 'Hf', 'ADI']
columns_to_correlate_with=['ICHN100_b100mean_100','FM100_b100mean_100',  'DN100_b100mean_100',
        'DI100_b100mean_100', 'AD100_b100mean_100',  'COB100_b100mean_100','brillo100_b100mean_100',
        'SLAVI100_b100mean_100', 'SL100_b100mean_100', 'NDMI100_b100mean_100','llanos100_b100mean_100']

correlation_df, h_value_df, p_value_df=calculate_correlation_kruskal(merged_df,merged_df, columns_to_correlate, columns_to_correlate_with)
correlation_df

,ICHN100_b100mean_100,FM100_b100mean_100,DN100_b100mean_100,DI100_b100mean_100,AD100_b100mean_100,COB100_b100mean_100,brillo100_b100mean_100,SLAVI100_b100mean_100,SL100_b100mean_100,NDMI100_b100mean_100,llanos100_b100mean_100
NDSI,0.735555,0.719105,0.375362,0.450149,0.748503,0.699156,-0.677712,0.725830,-0.126351,0.728665,0.712976
Hf,-0.087527,-0.088768,-0.032383,-0.074935,-0.096021,-0.091806,0.065210,-0.126566,0.044511,-0.138603,-0.115192
ADI,0.438451,0.447665,0.196298,0.240402,0.494500,0.417953,-0.399165,0.367384,-0.153307,0.372208,0.375800


In [ ]:
X_train11 = grab_8_17[['site','EPS', 'BGNf', 'Hf', 'NDSI', 'rBA', 'ADI', 'AEI']]
rand=3
GM = GaussianMixture(n_components=3, random_state=rand, covariance_type='full').fit(X_train11.iloc[:,1:])
pred=GM.predict(X_train11.iloc[:,1:])
predproba=GM.predict_proba(X_train11.iloc[:,1:])
X_train11["pred"]=pred
X_train11['component1_prob'] = predproba[:, 0]
X_train11['component2_prob'] = predproba[:, 1]
X_train11['component3_prob'] = predproba[:, 2]
merged_df = pd.merge(X_train11, buffer, on="site")
columns_to_correlate= ['component1_prob', 'component2_prob', 'component3_prob']
columns_to_correlate_with=['ICHN100_b100mean_100','FM100_b100mean_100',  'DN100_b100mean_100',
        'DI100_b100mean_100', 'AD100_b100mean_100',  'COB100_b100mean_100','brillo100_b100mean_100',
        'SLAVI100_b100mean_100', 'SL100_b100mean_100', 'NDMI100_b100mean_100','llanos100_b100mean_100']

correlation_df, h_value_df, p_value_df=calculate_correlation_kruskal(merged_df,merged_df, columns_to_correlate, columns_to_correlate_with)
correlation_df

,ICHN100_b100mean_100,FM100_b100mean_100,DN100_b100mean_100,DI100_b100mean_100,AD100_b100mean_100,COB100_b100mean_100,brillo100_b100mean_100,SLAVI100_b100mean_100,SL100_b100mean_100,NDMI100_b100mean_100,llanos100_b100mean_100
component1_prob,-0.675,-0.685,-0.328,-0.393,-0.714,-0.660,0.623,-0.658,0.158,-0.657,-0.649
component2_prob,0.282,0.270,0.184,0.227,0.266,0.269,-0.272,0.296,0.062,0.320,0.277
component3_prob,0.547,0.557,0.239,0.284,0.588,0.538,-0.487,0.515,-0.176,0.507,0.521
